In [1]:
from datasets import load_dataset

In [2]:
#dataset = load_dataset('json',data_files=['train.json','validation.json'])
dataset = load_dataset('json',data_files={'train':'../Datasets/train_authorities_bisnode.json','test':'../Datasets/test_authorities_bisnode.json'})

Using custom data configuration default-5573d7745b78f4ba
Reusing dataset json (C:\Users\liuyw\.cache\huggingface\datasets\json\default-5573d7745b78f4ba\0.0.0\d75ead8d5cfcbe67495df0f89bd262f0023257fbbbd94a730313295f3d756d50)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
dataset_input = load_dataset('json',data_files={'input':'../Datasets/input_authorities_bisnode.json'})

Using custom data configuration default-9c7c867de4386336
Reusing dataset json (C:\Users\liuyw\.cache\huggingface\datasets\json\default-9c7c867de4386336\0.0.0\d75ead8d5cfcbe67495df0f89bd262f0023257fbbbd94a730313295f3d756d50)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
dataset["train"][0:24]

{'idx': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23],
 'labels': [1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'sentence1': ['COL name VAL Landkreistag Saarland',
  'COL name VAL Weber Entec GmbH & Co. KG',
  'COL name VAL ec4u data experts gmbh',
  'COL name VAL Heinlein Support GmbH',
  'COL name VAL International Acetylene Association',
  'COL name VAL Verband Schweizerischer Elektrizitätswerke',
  'COL name VAL Kanada. Statistics Canada. Manufacturing and Primary Industries Division',
  'COL name VAL Hamburg. Gewerbeaufsichtsamt',
  'COL name VAL Museu de História Natural (Belo Horizonte)',
  'COL name VAL Weltgesundheitsorganisation. Expert Committee on Filariasis',
  'COL name VAL Sanshi-Shikenjō',
  'COL name VAL Landesverband für Obstbau, Garten und Landschaft Baden-Württemberg',
  'COL name VAL Müller und Gräf

In [56]:
dataset_input

DatasetDict({
    input: Dataset({
        features: ['idx', 'sentence1', 'sentence2'],
        num_rows: 4
    })
})

In [57]:
from transformers import AutoTokenizer

In [58]:
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at C:\Users\liuyw/.cache\huggingface\transformers\a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/resolve/main/vocab.txt from cache at C:\Users

In [59]:
def tokenize_function(example):
    return tokenizer(example["sentence1"],example["sentence2"],padding="max_length",truncation=True,max_length=128)

In [60]:
tokenized_datasets = dataset.map(tokenize_function,batched=True)
print(tokenized_datasets.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

{'train': ['attention_mask', 'idx', 'input_ids', 'labels', 'sentence1', 'sentence2', 'token_type_ids'], 'test': ['attention_mask', 'idx', 'input_ids', 'labels', 'sentence1', 'sentence2', 'token_type_ids']}


In [61]:
tokenized_datasets_input = dataset_input.map(tokenize_function,batched=True)
print(tokenized_datasets_input.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

{'input': ['attention_mask', 'idx', 'input_ids', 'sentence1', 'sentence2', 'token_type_ids']}


In [62]:
from transformers import DataCollatorWithPadding

In [63]:
data_collator = DataCollatorWithPadding(tokenizer)

In [64]:
from transformers import BertForSequenceClassification

In [65]:
model = BertForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at C:\Users\liuyw/.cache\huggingface\transformers\a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/main/pytorch_model.bin from c

In [66]:
from transformers import TrainingArguments

In [67]:
training_args = TrainingArguments("test-trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [68]:
from transformers import Trainer

trainer = Trainer(
        model,
        training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
        data_collator=data_collator,
        tokenizer=tokenizer,
        )
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running training *****
  Num examples = 24
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 9




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=9, training_loss=0.3258752293056912, metrics={'train_runtime': 21.9001, 'train_samples_per_second': 3.288, 'train_steps_per_second': 0.411, 'total_flos': 4735998996480.0, 'train_loss': 0.3258752293056912, 'epoch': 3.0})

In [69]:
predictions = trainer.predict(tokenized_datasets['test'])
print(predictions.predictions.shape,predictions.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running Prediction *****
  Num examples = 14
  Batch size = 8


(14, 2) (14,)


In [70]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

preds = np.argmax(predictions.predictions,axis=1)

metric.compute(predictions=preds,references=predictions.label_ids)

{'f1': 0.0}

In [71]:
predictions_input = trainer.predict(tokenized_datasets_input['input'])
print(predictions.predictions.shape,predictions.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running Prediction *****
  Num examples = 4
  Batch size = 8


(14, 2) (14,)


In [72]:
preds = np.argmax(predictions_input.predictions,axis=1)
preds

array([0, 0, 0, 0], dtype=int64)

In [73]:
with open('../Datasets/final_Bert_authorities_bisnode.txt', 'w') as f:
    for item in preds:
        f.write("%s\n" % item)